# 1、ChatMessageHistory的使用

场景1：记忆存储


In [1]:
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import PromptTemplate

# 1、ChatMessageHistory的实例化

history = ChatMessageHistory()

# 2、添加相关的消息进行存储

history.add_user_message("你好")

history.add_ai_message("很高兴认识你")

# 3、打印存储的消息
print(history.messages)

[HumanMessage(content='你好', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={})]


场景2：对接LLM



In [2]:
# 1、获取大模型
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

# 创建大模型实例
llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from langchain.memory import ChatMessageHistory

# 1、ChatMessageHistory的实例化

history = ChatMessageHistory()

# 2、添加相关的消息进行存储
history.add_user_message("你好")
history.add_ai_message("很高兴认识你")
history.add_user_message("帮我计算1 + 2 * 3 = ？")

response = llm.invoke(history.messages)
print(response.content)

根据数学运算的优先级，先进行乘法再进行加法。因此，计算过程如下：

1 + 2 * 3 = 1 + 6 = 7

所以，1 + 2 * 3 = 7。


# 2、ConversationBufferMemory的使用

举例1：以字符串的方式返回存储的信息

In [3]:
from langchain.memory import ConversationBufferMemory

# 1、ConversationBufferMemory的实例化
memory = ConversationBufferMemory()

# 2、存储相关的消息
# inputs对应的就是用户消息，outputs对应的就是ai消息
memory.save_context(inputs={"human": "你好，我叫小明"}, outputs={"ai": "很高兴认识你"})
memory.save_context(inputs={"input": "帮我回答一下1+2*3=?"}, outputs={"output": "7"})

# 3、获取存储的信息
print(memory.load_memory_variables({}))

#说明：返回的字典结构的key叫history.

{'history': 'Human: 你好，我叫小明\nAI: 很高兴认识你\nHuman: 帮我回答一下1+2*3=?\nAI: 7'}


举例2：以消息列表的方式返回存储的信息

In [8]:
from langchain.memory import ConversationBufferMemory

# 1、ConversationBufferMemory的实例化
memory = ConversationBufferMemory(return_messages=True)

# 2、存储相关的消息
# inputs对应的就是用户消息，outputs对应的就是ai消息
memory.save_context(inputs={"human": "你好，我叫小明"}, outputs={"ai": "很高兴认识你"})
memory.save_context(inputs={"input": "帮我回答一下1+2*3=?"}, outputs={"output": "7"})

# 3、获取存储的信息
#返回消息列表的方式1：
print(memory.load_memory_variables({}))

print("\n")

#返回消息列表的方式2：
print(memory.chat_memory.messages)

#说明：返回的字典结构的key叫history.

{'history': [HumanMessage(content='你好，我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}), HumanMessage(content='帮我回答一下1+2*3=?', additional_kwargs={}, response_metadata={}), AIMessage(content='7', additional_kwargs={}, response_metadata={})]}


[HumanMessage(content='你好，我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}), HumanMessage(content='帮我回答一下1+2*3=?', additional_kwargs={}, response_metadata={}), AIMessage(content='7', additional_kwargs={}, response_metadata={})]


举例3：结合大模型、提示词模板的使用（PromptTemplate）

In [12]:
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate

# 1、创建大模型实例
llm = ChatOpenAI(model="gpt-4o-mini")

# 2、提供提示词模板
prompt_template = PromptTemplate.from_template(
    template="""
    你可以与人类对话。

当前对话历史: {history}

人类问题: {question}

回复:
"""
)

# 3、提供memory实例
memory = ConversationBufferMemory()

# 4、提供Chain
chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory)

response = chain.invoke({"question": "你好，我的名字叫小明"})
print(response)

{'question': '你好，我的名字叫小明', 'history': '', 'text': '你好，小明！很高兴认识你。有什么我可以帮助你的吗？'}


In [13]:
response = chain.invoke({"question": "我叫什么名字呢？"})
print(response)

{'question': '我叫什么名字呢？', 'history': 'Human: 你好，我的名字叫小明\nAI: 你好，小明！很高兴认识你。有什么我可以帮助你的吗？', 'text': '你叫小明。很高兴再次和你交流！还有其他问题或者想聊的内容吗？'}


举例4：基于举例3，显式的设置meory的key的值


In [7]:
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate

# 1、创建大模型实例
llm = ChatOpenAI(model="gpt-4o-mini")

# 2、提供提示词模板
prompt_template = PromptTemplate.from_template(
    template="""
    你可以与人类对话。

当前对话历史: {chat_history}

人类问题: {question}

回复:
"""
)

# 3、提供memory实例
memory = ConversationBufferMemory(memory_key="chat_history")

# 4、提供Chain
chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory)

response = chain.invoke({"question": "你好，我的名字叫小明"})
print(response)

{'question': '你好，我的名字叫小明', 'chat_history': '', 'text': '你好，小明！很高兴认识你。有任何问题或者想聊的话题吗？'}


In [15]:
response = chain.invoke({"question": "我叫什么名字呢？"})
print(response)

{'question': '我叫什么名字呢？', 'chat_history': 'Human: 你好，我的名字叫小明\nAI: 你好，小明！很高兴认识你。有什么我可以帮助你的吗？', 'text': '你叫小明。很高兴再次见到你！有什么想聊的呢？'}


举例5：结合大模型、提示词模板的使用（ChatPromptTemplate）

In [16]:
# 1.导入相关包
from langchain_core.messages import SystemMessage
from langchain.chains.llm import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder,ChatPromptTemplate,HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI


# 2.创建LLM
llm = ChatOpenAI(model_name='gpt-4o-mini')

# 3.创建Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system","你是一个与人类对话的机器人。"),
    MessagesPlaceholder(variable_name='history'),
    ("human","问题：{question}")
])

# 4.创建Memory
memory = ConversationBufferMemory(return_messages=True)
# 5.创建LLMChain
llm_chain = LLMChain(prompt=prompt,llm=llm, memory=memory)

# 6.调用LLMChain
res1 = llm_chain.invoke({"question": "中国首都在哪里？"})
print(res1,end="\n\n")



{'question': '中国首都在哪里？', 'history': [HumanMessage(content='中国首都在哪里？', additional_kwargs={}, response_metadata={}), AIMessage(content='中国的首都位于北京。', additional_kwargs={}, response_metadata={})], 'text': '中国的首都位于北京。'}



In [17]:
res2 = llm_chain.invoke({"question": "我刚刚问了什么"})
print(res2)


{'question': '我刚刚问了什么', 'history': [HumanMessage(content='中国首都在哪里？', additional_kwargs={}, response_metadata={}), AIMessage(content='中国的首都位于北京。', additional_kwargs={}, response_metadata={}), HumanMessage(content='我刚刚问了什么', additional_kwargs={}, response_metadata={}), AIMessage(content='你刚刚问了中国的首都在哪里。', additional_kwargs={}, response_metadata={})], 'text': '你刚刚问了中国的首都在哪里。'}


# 3、ConversationChain的使用

举例1：以PromptTemplate为例

In [19]:
from langchain.chains.conversation.base import ConversationChain
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate

# 1、创建大模型实例
llm = ChatOpenAI(model="gpt-4o-mini")

# 2、提供提示词模板
prompt_template = PromptTemplate.from_template(
    template="""
    你可以与人类对话。

当前对话历史: {history}

人类问题: {input}

回复:
"""
)

# # 3、提供memory实例
# memory = ConversationBufferMemory()
#
# # 4、提供Chain
# chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory)

# 3、创建ConversationChain的实例
chain = ConversationChain(llm = llm, prompt=prompt_template)

response = chain.invoke({"input": "你好，我的名字叫小明"})
print(response)

{'input': '你好，我的名字叫小明', 'history': '', 'response': '你好，小明！很高兴认识你。你今天过得怎么样？'}


In [20]:
response = chain.invoke({"input": "我的名字叫什么？"})
print(response)

{'input': '我的名字叫什么？', 'history': 'Human: 你好，我的名字叫小明\nAI: 你好，小明！很高兴认识你。你今天过得怎么样？', 'response': '你的名字叫小明。'}


举例2：使用默认提供的提示词模板



In [21]:
from langchain.chains.conversation.base import ConversationChain
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate

# 1、创建大模型实例
llm = ChatOpenAI(model="gpt-4o-mini")

# 2、提供提示词模板
# prompt_template = PromptTemplate.from_template(
#     template="""
#     你可以与人类对话。
#
# 当前对话历史: {history}
#
# 人类问题: {input}
#
# 回复:
# """
# )

# # 3、提供memory实例
# memory = ConversationBufferMemory()
#
# # 4、提供Chain
# chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory)

# 3、创建ConversationChain的实例（内部提供了默认的提示词模板。而此模板中的变量是{input}、{history}
chain = ConversationChain(llm = llm)

response = chain.invoke({"input": "你好，我的名字叫小明"})
print(response)

{'input': '你好，我的名字叫小明', 'history': '', 'response': '你好，小明！很高兴认识你！我是一种人工智能助手，随时准备帮助你。你今天怎么样？有什么特别想聊的吗？'}


In [22]:
response = chain.invoke({"input": "我的名字叫什么？"})
print(response)

{'input': '我的名字叫什么？', 'history': 'Human: 你好，我的名字叫小明\nAI: 你好，小明！很高兴认识你！我是一种人工智能助手，随时准备帮助你。你今天怎么样？有什么特别想聊的吗？', 'response': '你的名字叫小明！我记得你刚刚告诉我这个信息。你想聊些什么，或者有什么问题需要我帮助的吗？'}


# 4、ConversationBufferWindowMemory的使用

举例1：


In [25]:
# 1.导入相关包
from langchain.memory import ConversationBufferWindowMemory

# 2.实例化ConversationBufferWindowMemory对象，设定窗口阈值
memory = ConversationBufferWindowMemory(k=1)
# 3.保存消息
memory.save_context({"input": "你好"}, {"output": "怎么了"})
memory.save_context({"input": "你是谁"}, {"output": "我是AI助手"})
memory.save_context({"input": "你的生日是哪天？"}, {"output": "我不清楚"})
# 4.读取内存中消息（返回消息内容的纯文本）
print(memory.load_memory_variables({}))

{'history': 'Human: 你的生日是哪天？\nAI: 我不清楚'}


举例2：返回消息构成的上下文记忆


In [26]:
# 1.导入相关包
from langchain.memory import ConversationBufferWindowMemory

# 2.实例化ConversationBufferWindowMemory对象，设定窗口阈值
memory = ConversationBufferWindowMemory(k=2, return_messages=True)
# 3.保存消息
memory.save_context({"input": "你好"}, {"output": "怎么了"})
memory.save_context({"input": "你是谁"}, {"output": "我是AI助手小智"})
memory.save_context({"input": "初次对话，你能介绍一下你自己吗？"}, {"output": "当然可以了。我是一个无所不能的小智。"})
# 4.读取内存中消息（返回消息内容的纯文本）
print(memory.load_memory_variables({}))

{'history': [HumanMessage(content='你是谁', additional_kwargs={}, response_metadata={}), AIMessage(content='我是AI助手小智', additional_kwargs={}, response_metadata={}), HumanMessage(content='初次对话，你能介绍一下你自己吗？', additional_kwargs={}, response_metadata={}), AIMessage(content='当然可以了。我是一个无所不能的小智。', additional_kwargs={}, response_metadata={})]}


举例3：结合llm、chain的使用


In [30]:
from langchain.memory import ConversationBufferWindowMemory
# 1.导入相关包
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain

# 2.定义模版
template = """以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：
{history}
Human: {question}
AI:"""

# 3.定义提示词模版
prompt_template = PromptTemplate.from_template(template)

# 4.创建大模型
llm = ChatOpenAI(model="gpt-4o-mini")

# 5.实例化ConversationBufferWindowMemory对象，设定窗口阈值
memory = ConversationBufferWindowMemory(k=1)

# 6.定义LLMChain
conversation_with_summary = LLMChain(
    llm=llm,
    prompt=prompt_template,
    memory=memory,
    #verbose=True,
)

# 7.执行链（第一次提问）
respon1 = conversation_with_summary.invoke({"question":"你好，我是孙小空"})
print(respon1)
# 8.执行链（第二次提问）
respon2 =conversation_with_summary.invoke({"question":"我还有两个师弟，一个是猪小戒，一个是沙小僧"})
print(respon2)
# 9.执行链（第三次提问）
respon3 =conversation_with_summary.invoke({"question":"我今年高考，竟然考上了1本"})
print(respon3)
# 10.执行链（第四次提问）
respon4 =conversation_with_summary.invoke({"question":"我叫什么名字？"})
print(respon4)

{'question': '你好，我是孙小空', 'history': '', 'text': '你好，孙小空！很高兴见到你！你今天过得怎么样？有什么我可以帮助你的吗？'}
{'question': '我还有两个师弟，一个是猪小戒，一个是沙小僧', 'history': 'Human: 你好，我是孙小空\nAI: 你好，孙小空！很高兴见到你！你今天过得怎么样？有什么我可以帮助你的吗？', 'text': '哇，猪小戒和沙小僧听起来很有趣！他们是不是你们团队里的搞笑角色呢？你们平时都一起做些什么呢？'}
{'question': '我今年高考，竟然考上了1本', 'history': 'Human: 我还有两个师弟，一个是猪小戒，一个是沙小僧\nAI: 哇，猪小戒和沙小僧听起来很有趣！他们是不是你们团队里的搞笑角色呢？你们平时都一起做些什么呢？', 'text': '太厉害了！恭喜你考上了一本！这是一个很大的成就，你一定付出了很多努力。你想要学什么专业呢？或者有没有什么学校是你特别想去的？'}
{'question': '我叫什么名字？', 'history': 'Human: 我今年高考，竟然考上了1本\nAI: 太厉害了！恭喜你考上了一本！这是一个很大的成就，你一定付出了很多努力。你想要学什么专业呢？或者有没有什么学校是你特别想去的？', 'text': '抱歉，我不知道你的名字。你愿意告诉我吗？或者我们可以聊聊其他你感兴趣的事情！'}


举例4：修改举例3中的参数k


In [31]:
from langchain.memory import ConversationBufferWindowMemory
# 1.导入相关包
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain

# 2.定义模版
template = """以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：
{history}
Human: {question}
AI:"""

# 3.定义提示词模版
prompt_template = PromptTemplate.from_template(template)

# 4.创建大模型
llm = ChatOpenAI(model="gpt-4o-mini")

# 5.实例化ConversationBufferWindowMemory对象，设定窗口阈值
memory = ConversationBufferWindowMemory(k=3)

# 6.定义LLMChain
conversation_with_summary = LLMChain(
    llm=llm,
    prompt=prompt_template,
    memory=memory,
    #verbose=True,
)

# 7.执行链（第一次提问）
respon1 = conversation_with_summary.invoke({"question":"你好，我是孙小空"})
# print(respon1)
# 8.执行链（第二次提问）
respon2 =conversation_with_summary.invoke({"question":"我还有两个师弟，一个是猪小戒，一个是沙小僧"})
# print(respon2)
# 9.执行链（第三次提问）
respon3 =conversation_with_summary.invoke({"question":"我今年高考，竟然考上了1本"})
# print(respon3)
# 10.执行链（第四次提问）
respon4 =conversation_with_summary.invoke({"question":"我叫什么名字？"})
print(respon4)

{'question': '我叫什么名字？', 'history': 'Human: 你好，我是孙小空\nAI: 你好，孙小空！很高兴认识你。你今天过得怎么样？有什么想聊的内容吗？\nHuman: 我还有两个师弟，一个是猪小戒，一个是沙小僧\nAI: 很高兴认识你的师弟们！猪小戒和沙小僧的名字听起来很有趣，似乎有些文艺或是神话背景。你们是不是一起学习或者修炼什么特别的技能呢？如果有趣的故事或者共同经历，欢迎分享！\nHuman: 我今年高考，竟然考上了1本\nAI: 太棒了，孙小空！恭喜你考上了一本大学！这是一个很大的成就，你一定为自己感到骄傲。你打算学什么专业呢？或者你对未来有什么计划吗？', 'text': '你叫孙小空！如果你有其他问题或者想聊的话题，请随时告诉我！'}
